In [2]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv
from pathlib import Path

In [4]:
# Get the absolute path to the .env file
env_path = Path(os.getcwd()).parent / ".env"

# Load the .env file
load_dotenv(dotenv_path=env_path)

True

In [5]:
# Get environment variable
AZURE_AI_SERVICE_NAME = os.getenv("AZURE_AI_SERVICE_NAME")

# Create the token provider
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o",
    api_version="2024-05-01-preview",
    model="gpt-4o",
    azure_endpoint=f"https://{AZURE_AI_SERVICE_NAME}.openai.azure.com/",
    azure_ad_token_provider=token_provider,
)

In [6]:
# Create the developer agent.
developer_agent = AssistantAgent(
    name="developer",
    model_client=az_model_client,
    system_message="You are a dotnet developer that uses C# as language."
)

# Create the tech-lead agent.
techlead_agent = AssistantAgent(
    name="techlead",
    model_client=az_model_client,
    system_message="You are a dotnet technical lead. Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed."
)

In [7]:
# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([developer_agent, techlead_agent], termination_condition=text_termination)

In [ ]:
task = "Create a File Service that does all CRUD operations with files."
await Console(team.run_stream(task=task))